In [22]:
import pandas as pd
df = pd.read_csv('enhanced_parsed_shot_events.csv')
df.columns

Index(['season', 'gameId', 'eventId', 'period', 'timeInPeriod', 'eventType',
       'teamId', 'shooter', 'goalie', 'HomevsAway', 'shotType', 'emptyNetAway',
       'emptyNetHome', 'powerplayHome', 'powerplayAway', 'coordinates',
       'result', 'homeTeamId', 'awayTeamId', 'emptyNetGoal', 'offensiveSide',
       'xCoord', 'yCoord', 'shotDistance', 'shotAngle', 'lastEventType',
       'lastEventXCoord', 'lastEventYCoord', 'timeElapsedSinceLastEvent',
       'distanceFromLastEvent', 'rebound', 'changeInShotAngle', 'speed'],
      dtype='object')

In [23]:
df = df.dropna(subset=['shotDistance', 'shotAngle', 'emptyNetGoal', 'result'])
df.isna().sum()

season                            0
gameId                            0
eventId                           0
period                            0
timeInPeriod                      0
eventType                         0
teamId                            0
shooter                           0
goalie                         1734
HomevsAway                        0
shotType                         33
emptyNetAway                      0
emptyNetHome                      0
powerplayHome                     0
powerplayAway                     0
coordinates                       0
result                            0
homeTeamId                        0
awayTeamId                        0
emptyNetGoal                      0
offensiveSide                     0
xCoord                            0
yCoord                            0
shotDistance                      0
shotAngle                         0
lastEventType                     0
lastEventXCoord               12445
lastEventYCoord             

In [25]:
import math
import pandas as pd
import ast

def shotAngle_column(df):
    def abs_angle(row):
        # Parse coordinates as a tuple if stored as a string
        angle = row['shotAngle']
        if angle < 0:
            angle = angle * (-1)
        
        
        # Return both distance and angle
        return angle
    
    # Apply the function to each row in the DataFrame and create the new columns
    df['shotAngle'] = df.apply(abs_angle, axis=1)
    
    return df

# Exemple d'utilisation
df = shotAngle_column(df)


In [26]:
df

,season,gameId,eventId,period,timeInPeriod,eventType,teamId,shooter,goalie,HomevsAway,...,shotDistance,shotAngle,lastEventType,lastEventXCoord,lastEventYCoord,timeElapsedSinceLastEvent,distanceFromLastEvent,rebound,changeInShotAngle,speed
0,20162017,2016020001,8,1,01:11,shot-on-goal,10,Mitch Marner,Craig Anderson,away,...,13.000000,22.619865,blocked-shot,-61.0,11.0,1.0,17.088007,False,NaN,17.088007
1,20162017,2016020001,11,1,02:53,shot-on-goal,9,Chris Kelly,Frederik Andersen,home,...,13.341664,77.005383,giveaway,54.0,-5.0,5.0,36.715120,False,NaN,7.343024
2,20162017,2016020001,15,1,04:01,shot-on-goal,9,Cody Ceci,Frederik Andersen,home,...,76.157731,29.931512,missed-shot,-72.0,0.0,18.0,102.318131,False,NaN,5.684341
3,20162017,2016020001,16,1,04:46,shot-on-goal,9,Erik Karlsson,Frederik Andersen,home,...,57.974132,14.995079,missed-shot,77.0,-2.0,19.0,45.880279,False,NaN,2.414752
4,20162017,2016020001,24,1,06:46,shot-on-goal,10,Martin Marincin,Craig Anderson,away,...,61.717096,26.980231,hit,47.0,34.0,16.0,81.221918,False,NaN,5.076370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385071,20202021,2020030415,692,3,16:19,shot-on-goal,14,Blake Coleman,Carey Price,home,...,43.863424,43.152390,shot-on-goal,-71.0,35.0,15.0,128.097619,True,30.813302,8.539841
385072,20202021,2020030415,693,3,17:13,shot-on-goal,14,Mikhail Sergachev,Carey Price,home,...,45.276926,43.210089,hit,67.0,40.0,9.0,71.847060,False,NaN,7.983007
385073,20202021,2020030415,697,3,19:01,shot-on-goal,8,Tyler Toffoli,Andrei Vasilevskiy,away,...,46.173586,17.650124,faceoff,-69.0,-22.0,25.0,25.298221,False,NaN,1.011929
385074,20202021,2020030415,698,3,19:10,shot-on-goal,8,Cole Caufield,Andrei Vasilevskiy,away,...,12.369317,14.036243,shot-on-goal,-45.0,-14.0,9.0,33.837849,True,3.613881,3.759761


In [27]:
# Séparer les données en ensembles d'entraînement et de test
df_train = df[df['season'] != 20202021]  # Données d'entraînement pour toutes les saisons sauf 2020-2021
df_test = df[df['season'] == 20202021]   # Données de test uniquement pour la saison 2020-2021

# Vérification des dimensions
print("Taille de l'ensemble d'entraînement :", df_train.shape)
print("Taille de l'ensemble de test :", df_test.shape)

Taille de l'ensemble d'entraînement : (325375, 33)
Taille de l'ensemble de test : (57297, 33)


In [28]:
# Remplacer 'no goal' par 0 et 'goal' par 1 dans la colonne 'result'
df_train['result'] = df_train['result'].replace({'no goal': 0, 'goal': 1})
df_test['result'] = df_test['result'].replace({'no goal': 0, 'goal': 1})
# Garder uniquement les colonnes sélectionnées
df_train = df_train[['shotDistance', 'shotAngle', 'emptyNetGoal', 'result']]
df_test = df_test[['shotDistance', 'shotAngle', 'emptyNetGoal', 'result']]



C:\Users\Admin\AppData\Local\Temp\ipykernel_11704\913144208.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['result'] = df_train['result'].replace({'no goal': 0, 'goal': 1})
C:\Users\Admin\AppData\Local\Temp\ipykernel_11704\913144208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['result'] = df_train['result'].replace({'no goal': 0, 'goal': 1})
C:\Users\Admin\AppData\Local\Temp\ipykernel_11704\913144208.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed i

In [29]:
df_train.to_csv('train_data.csv',index=False)
df_train

,shotDistance,shotAngle,emptyNetGoal,result
0,13.000000,22.619865,0,0
1,13.341664,77.005383,0,0
2,76.157731,29.931512,0,0
3,57.974132,14.995079,0,0
4,61.717096,26.980231,0,0
...,...,...,...,...
327337,38.418745,38.659808,0,0
327338,8.544004,20.556045,0,0
327339,55.326305,6.225829,0,0
327340,58.855756,9.782407,0,0


In [30]:
df_test.to_csv('test_data.csv',index=False)
df_test

,shotDistance,shotAngle,emptyNetGoal,result
327342,32.649655,62.650124,0,0
327343,47.169906,32.005383,0,0
327344,48.270074,39.957549,0,0
327345,60.307545,40.292148,0,0
327346,36.878178,12.528808,0,0
...,...,...,...,...
385071,43.863424,43.152390,0,0
385072,45.276926,43.210089,0,0
385073,46.173586,17.650124,0,0
385074,12.369317,14.036243,0,0
